<a href="https://colab.research.google.com/github/theclosedbook/TensorFlow/blob/main/04_Transfer_Learning_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with TensorFlow Part 1 : Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.
There are two main benefits:

1. Can leverage an existing neural network arechitecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture whi9ch has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.

In [1]:
# Are we using a GPU
!nvidia-smi

Mon Feb 12 15:14:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Downlaod and become one with the data

In [7]:
# Get the data
import zipfile

# Downlaod the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2024-02-12 16:41:58--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  21.9MB/s    in 8.3s    

2024-02-12 16:42:06 (19.4 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
# How many images in each folder
import os

# Walk through the directories
for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

### creating a dataloader (preparing the data)
We'll use `ImageDataGenerator` class to load in our images

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE=32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images:")
train_data_10_precent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")

print("Testing Data:")

test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                        target_size=IMAGE_SHAPE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical")

Training Images:
Found 750 images belonging to 10 classes.
Testing Data:
Found 2500 images belonging to 10 classes.


### Setting up callbacks(things to run whilst our model train)

Callbacks are extra functionallity you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Training experiments with the TensorBoard callback.
* Model checkpoint with the ModelCheckpoint callback.
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback.

In [17]:
# Create TensorBoard callback(functionized becasue we need to create a new one for each model)
import tensorflow as tf
import datetime
def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback

## Creating models using TensorFlow Hub

In the past we have used Tensorflow to create our own models layer by layer from scratch.

Now we're going to do a similar process, except the majority of our model's layers are going to come from TensorFlow Hub.

We can access pretrained models on:https://tfhub.dev/

In [10]:
# Lets compare the following two models

resnet_url = "https://www.kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-feature-vector/versions/2"

efficientnet_url = "https://www.kaggle.com/models/tensorflow/efficientnet/frameworks/TensorFlow2/variations/b0-feature-vector/versions/1"

In [2]:
# Import dependencies

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [3]:
# Let's make a create_model() function to create a model from URL

def create_model(model_url,num_classes=10):
  # Download a pretrained model and save as a keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,)) # Freeze the already learned patterns

  # Create our own model
  model=tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes,activation="softmax", name="output_layer")
  ])

  return model

## Creating and tetsing ResNet TensorFlow Hub Feature Extraction model

In [11]:
# Create a Resnet model

resnet_model = create_model(resnet_url,
                            num_classes=train_data_10_precent.num_classes)

In [12]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 2048)              23564800  
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23585290 (89.97 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [13]:
# Compile the model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [19]:
# Fit our model
resnet_history = resnet_model.fit(train_data_10_precent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_precent),
                                  validation_data=test_data_10_percent,
                                  validation_steps=len(test_data_10_percent),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50V2")])

Saving TensorBoard log files to : tensorflow_hub/resnet50V2/20240212-165626
Epoch 1/5
24/24 [==============================] - 11s 455ms/step - loss: 0.2553 - accuracy: 0.9587 - val_loss: 0.6359 - val_accuracy: 0.8008
Epoch 2/5
24/24 [==============================] - 13s 547ms/step - loss: 0.2140 - accuracy: 0.9680 - val_loss: 0.6335 - val_accuracy: 0.7920
Epoch 3/5
24/24 [==============================] - 12s 494ms/step - loss: 0.1914 - accuracy: 0.9760 - val_loss: 0.6330 - val_accuracy: 0.7944
Epoch 4/5
24/24 [==============================] - 11s 477ms/step - loss: 0.1607 - accuracy: 0.9853 - val_loss: 0.6225 - val_accuracy: 0.8048
Epoch 5/5
24/24 [==============================] - 11s 459ms/step - loss: 0.1396 - accuracy: 0.9920 - val_loss: 0.6270 - val_accuracy: 0.7968


In [ ]:
# Create afunction for our loss curves

import matplotlib.pyplot as plt

def plot_loss_curves(history):
  loss=history.history["loss"]
  val_loss = history.history["val_loss"]